# Nanodegree Engenheiro de Machine Learning
## Aprendizagem Supervisionada
## Projeto 2: Construindo um Sistema de Intervenção para Estudantes

Solução de Fernando Martinelli Ramacciotti

Disponível [neste link](https://github.com/fernandoramacciotti/udacity_nanodegree_machine-learning-engineer/tree/master/projects/student_intervention)

Bem-vindo ao segundo projeto do Nanodegree de Machine Learning! Neste Notebook, alguns templates de código já foram fornecidos, e será o seu trabalho implementar funcionalidades necessárias para completar este projeto com êxito. Seções que começam com **'Implementação'** no cabeçalho indicam que o bloco de código que se segue precisará de funcionalidades adicionais que você deve fornecer. Instruções serão providenciadas para cada seção e as especificações para cada implementação estarão marcadas no bloco de código com o comando `'TODO'`. Tenha certeza de ler atentamente todas as instruções!

Além do código implementado, haverá questões relacionadas ao projeto e à implementação que você deve responder. Cada seção em que você tem que responder uma questão será antecedida de um cabeçalho **'Questão X'**. Leia atentamente cada questão e escreva respostas completas nas caixas de texto subsequentes que começam com **'Resposta: '**. O projeto enviado será avaliado baseado nas respostas para cada questão e a implementação que você forneceu.  

>**Nota:** Células de código e Markdown podem ser executadas utilizando o atalho de teclado **Shift + Enter**. Além disso, as células Markdown podem ser editadas, um clique duplo na célula entra no modo de edição.

### Questão 1 - Classificação versus Regressão
*Seu objetivo neste projeto é identificar estudantes que possam precisar de intervenção antecipada antes de serem reprovados. Que tipo de problema de aprendizagem supervisionada é esse: classificação ou regressão? Por quê?*

**Resposta: **
Este é um problema de classificação, pois devemos prever se o estudante precisa de intervenção ou não - uma decisão binária. Não podemos transformar isso em uma regressão, pois não é um espaço contínuo e não pode assumir infinitos valores, aqui é precisa de intervenção sim ou não.

## Observando os Dados
Execute a célula de código abaixo para carregar as bibliotecas de Python necessárias e os dados sobre os estudantes. Note que a última coluna desse conjunto de dados, `'passed'`, será nosso rótulo alvo (se o aluno foi ou não aprovado). As outras colunas são atributos sobre cada aluno.

In [1]:
# Importar bibliotecas
from __future__ import division
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Ler os dados dos estudantes
student_data = pd.read_csv('./input/student-data.csv')
print "Os dados dos estudantes foram lidos com êxito!"

Os dados dos estudantes foram lidos com êxito!


### Implementação: Observando os Dados
Vamos começar observando o conjunto de dados para determinar quantos são os estudantes sobre os quais temos informações e entender a taxa de graduação entre esses estudantes. Na célula de código abaixo, você vai precisar calcular o seguinte:
- O número total de estudantes, `n_students`.
- O número total de atributos para cada estudante, `n_features`.
- O número de estudantes aprovados, `n_passed`.
- O número de estudantes reprovados, `n_failed`.
- A taxa de graduação da classe, `grad_rate`, em porcentagem (%).


In [2]:
# vamos ver a cara dos dados, apenas para saber o tipo de cada feature
student_data.head(5)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,passed
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,6,no
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,4,no
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,2,3,3,10,yes
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3,2,2,1,1,5,2,yes
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4,3,2,1,2,5,4,yes


In [3]:
# TODO: Calcule o número de estudante
n_students = student_data.shape[0] 
# a propriedade shape de um df retorna uma tupla (n_linhas, n_colunas)
# geralmente temos as observações por linha e cada observação é um estudante

# TODO: Calcule o número de atributos
n_features = student_data.shape[1] - 1 # numero de colunas - 1, pois 'passed' é a variável alvo

# TODO: Calcule o número de alunos aprovados
n_passed = student_data[student_data.passed == 'yes'].shape[0] # numero de linhas do subset com passed == yes

# TODO: Calcule o número de alunos reprovados
n_failed = student_data[student_data.passed == 'no'].shape[0] # numero de linhas do subset com passed == no

# TODO: Calcule a taxa de graduação
# ainda nao checamos, mas pode haver um rotulo diferente de yes ou no (talvez missing data)
# portanto, faremos a taxa de graduacao sobre o total da soma de yes e no
grad_rate = n_passed / (n_passed + n_failed) * 100

# Imprima os resultados
print "Número total de estudantes: {}".format(n_students)
print "Número de atributos: {}".format(n_features)
print "Número de estudantes aprovados: {}".format(n_passed)
print "Número de estudantes reprovados: {}".format(n_failed)
print "Taxa de graduação: {:.2f}%".format(grad_rate)

Número total de estudantes: 395
Número de atributos: 30
Número de estudantes aprovados: 265
Número de estudantes reprovados: 130
Taxa de graduação: 67.09%


*disclaimer: temos de fato que o numero de estudantes aprovados + reprovados = numero total*

## Preparando os Dados
Nesta seção, vamos preparara os dados para modelagem, treinamento e teste.

### Identificar atributos e variáveis-alvo
É comum que os dados que você obteve contenham atributos não numéricos. Isso pode ser um problema, dado que a maioria dos algoritmos de machine learning esperam dados númericos para operar cálculos.

Execute a célula de código abaixo para separar os dados dos estudantes em atributos e variáveis-alvo e verificar se algum desses atributos é não numérico.

In [4]:
# Extraia as colunas dos atributo
feature_cols = list(student_data.columns[:-1])

# Extraia a coluna-alvo 'passed'
target_col = student_data.columns[-1] 

# Mostre a lista de colunas
print "Colunas de atributos:\n{}".format(feature_cols)
print "\nColuna-alvo: {}".format(target_col)

# Separe os dados em atributos e variáveis-alvo (X_all e y_all, respectivamente)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Mostre os atributos imprimindo as cinco primeiras linhas
print "\nFeature values:"
print X_all.head()

Colunas de atributos:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Coluna-alvo: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Pré-processar Colunas de Atributo

Como você pode ver, há muitas colunas não numéricas que precisam ser convertidas! Muitas delas são simplesmente `yes`/`no`, por exemplo, a coluna `internet`. É razoável converter essas variáveis em valores (binários) `1`/`0`.

Outras colunas, como `Mjob` e `Fjob`, têm mais do que dois valores e são conhecidas como variáveis categóricas. A maneira recomendada de lidar com esse tipo de coluna é criar uma quantidade de colunas proporcional aos possíveis valores (por exemplo, `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc), e assinalar `1` para um deles e `0` para todos os outros.

Essas colunas geradas são por vezes chamadas de _variáveis postiças_ (_dummy variables_), e nós iremos utilizar a função [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) para fazer essa conversão. Execute a célula de código abaixo para executar a rotina de pré-processamento discutida nesta seção.

In [5]:
def preprocess_features(X):
    ''' Pré-processa os dados dos estudantes e converte as variáveis binárias não numéricas em
        variáveis binárias (0/1). Converte variáveis categóricas em variáveis postiças. '''
    
    # Inicialize nova saída DataFrame
    output = pd.DataFrame(index = X.index)

    # Observe os dados em cada coluna de atributos 
    for col, col_data in X.iteritems():
        
        # Se o tipo de dado for não numérico, substitua todos os valores yes/no por 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # Se o tipo de dado for categórico, converta-o para uma variável dummy
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Reúna as colunas revisadas
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementação: Divisão dos Dados de Treinamento e Teste
Até agora, nós convertemos todos os atributos _categóricos_ em valores numéricos. Para o próximo passo, vamos dividir os dados (tanto atributos como os rótulos correspondentes) em conjuntos de treinamento e teste. Na célula de código abaixo, você irá precisar implementar o seguinte:
- Embaralhe aleatoriamente os dados (`X_all`, `y_all`) em subconjuntos de treinamento e teste.
  - Utilizar 300 pontos de treinamento (aproxidamente 75%) e 95 pontos de teste (aproximadamente 25%).
  - Estabelecer um `random_state` para as funções que você utiliza, se a opção existir.
  - Armazene os resultados em `X_train`, `X_test`, `y_train` e `y_test`.

In [6]:
# TODO: Importe qualquer funcionalidade adicional de que você possa precisar aqui
from sklearn.model_selection import train_test_split

# TODO: Estabeleça o número de pontos de treinamento
num_train = 300

# Estabeleça o número de pontos de teste
num_test = X_all.shape[0] - num_train

# TODO: Emabaralhe e distribua o conjunto de dados de acordo com o número de pontos de treinamento e teste abaixo
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=0)

# Mostre o resultado da distribuição
print "O conjunto de treinamento tem {} amostras.".format(X_train.shape[0])
print "O conjunto de teste tem {} amostras.".format(X_test.shape[0])

O conjunto de treinamento tem 300 amostras.
O conjunto de teste tem 95 amostras.


## Treinando e Avaliando Modelos
Nesta seção, você irá escolher 3 modelos de aprendizagem supervisionada que sejam apropriados para esse problema e que estejam disponíveis no `scikit-learn`. Primeiro você irá discutir o raciocínio por trás da escolha desses três modelos considerando suas vantagens e desvantagens e o que você sabe sobre os dados. Depois você irá ajustar o modelo a diferentes tamanhos de conjuntos de treinamento (com 100, 200 e 300 pontos) e medir a pontuação F<sub>1</sub>. Você vai precisar preencher três tabelas (uma para cada modelo) que mostrem o tamanho do conjunto de treinamento, o tempo de treinamento, o tempo de previsão e a pontuação F<sub>1</sub> no conjunto de treinamento.

**Os seguintes modelos de aprendizagem supervisionada estão atualmente disponíveis no **[`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html)** para você escolher:**
- Gaussian Naive Bayes (GaussianNB)
- Árvores de Decisão
- Métodos de agregação (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Método do gradiente estocástico (SGDC)
- Máquinas de vetores de suporte (SVM)
- Regressão logística

### Questão 2 - Aplicação dos Modelos
*Liste três modelos de aprendizagem supervisionada que são apropriadas para esse problema. Para cada modelo escolhido:*
- Descreva uma aplicação em mundo real na indústria em que o modelo pode ser aplicado. *(Talvez você precise fazer um pouco de pesquisa para responder essa questão – dê as devidas referências!)* 
- Quais são as vantagens do modelo; quando ele tem desempenho melhor? 
- Quais são as desvantagens do modelo, quando ele tem desempenho pior?
- O que faz desse modelo um bom candidato para o problema, considerando o que você sabe sobre os dados?

**Resposta: **

Os modelos escolhidos foram:

**1. Regressão Logística**

O modelo clássico de regressão logística sempre vem a tona quando nos deparamos com um problema de classificação. Ele pode ser, simplisticamente, pensado como uma regressão linear mas que o *output* é limitado estar dentro do intervalo [0, 1] e, portanto, introduz uma certa não-linearidade. O resultado pode ser interpretado como a probabilidade de pertencer a uma certa classe ou não, já que está entre 0 e 1.

Uma aplicação muito comum desse modelo é para *credit scoring*, isto é, análise de crédito para o banco, por exemplo, prever quais as chances do tomador do empréstimo não pagar a dívida. A [Deloitte](https://www2.deloitte.com/content/dam/Deloitte/global/Documents/Financial-Services/gx-be-aers-fsi-credit-scoring.pdf) dá um exemplo de como é feita a modelagem para esse uso e é muito comum nas áreas de crédito de grandes bancos e empresas anti-fraudes de varejo online.

O modelo performa bem quando não há colinearidade entre as variáveis preditoras, ou seja, quando há uma alta correlação entre as covariadas (*features*). Além disso, assume-se que as observações são independentes entre si. O modelo é geralmente estimado por máxima verossimilhança em vez de mínimos quadrados ordinários. Assim, se os nossos dados não forem representativos, eles enviesarão a nossa estimativa, pois esse método de otimização acha o valor dos parâmetros que sejam mais prováveis dado os dados.

Dado os nossos dados, devemos checar a colinearidade entre as covariadas e, caso haja, eliminar aquelas redundantes. Pode-se argumentar, também, que as nossas observações podem apresentar certo grau de dependência, uma vez que a performance de um aluno pode influenciar a de outro, devido a uma séries de fatores não observáveis, como o grau de amizade.

**2. K-Nearest Neighbours (KNN)**

O modelo de KNN é tanto simples quanto poderoso. A ideia é de classificar um dado ponto novo de acordo com os $K$ pontos mais próximos dele (dada uma métrica de distância) e a classificação é dada por votação. Assim, quanto maior $K$, mais suave é a fronteira de classificação. No entanto, no caso extremo de $K=1$ o erro de treinamento é muito baixo, mas *overfitting* é inevitável.

O algortimo é bastante usado na indústria, como por exemplo em algortimos de [sistemas recomendadores](http://glaros.dtc.umn.edu/gkhome/node/122). O modelo identifica padrões e hábitos de pessoas similares, por exemplo, e consegue recomendar produtos que possam interessar àquele consumidor.

O modelo tem como vantagens o baixíssimo custo de treinamento, mas altíssimo custo durante a fase de teste. Isso ocorre, pois durante a fase de teste, o algoritmo praticamente usa todos os pontos de treinamento para avaliar os $K$ pontos mais próximos dos pontos de teste em questão e esse custa aumenta com o tamanho de $K$.

Dado os nossos dados, ainda não exploramos as relações das *features*, mas o KNN lida bem com não-linearidades e é possível que nos deparemos com isso nos nossos dados. Avaliar o algoritmo nesse contexto, desde que o custo computacional não seja alto.

**3. Random Forest **

Modelos de agregação, de um modo geral, tem um excelente desempenho. O modelo de random forest pode ser usado em praticamente qualquer problema de classificação. Aplicações interessantes têm se revelado na medicina, como no artigo de [Casanova (2014)](http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0098587) sobre retinopatia diabética.

Uma grande vantagem do modelo de Random Forest é que, como no caso de árvores de regressão, não é necessário nenhum normalização ou pré-tratamento dos dados para rodá-lo. Eventualmente, tal pré-processamento pode melhorar o desempenho do modelo, mas não é estritamente necessário. Além disso, devido a randomização intrínseca do algoritmo, a seleção das *features* (*feature selection*) já é, de certa forma, feita.

Um ponto de atenção para o uso de tal algortimo é a interpretabilidade do modelo. É praticamente impossível saber ao certo como as diversas árvores foram construídas e isso é parte do sucesso do algortimo, a randomização do *subset* dos dados para cada árvore construída. Assim, para um modelo do nosso tipo, se desejamos ter um verdadeiro entendimento de causa e efeito a fim de desenvolver políticas para evitar já desde o início a desistência de alunos, random forest certamente não é a melhor das técnicas.

### Configuração
Execute a célula de código abaixo para inicializar três funções de ajuda que você pode utilizar para treinar e testar os três modelos de aprendizagem supervisionada que você escolheu acima. As funções são as seguintes:
- `train_classifier` - recebe como parâmetro um classificador e dados de treinamento e ajusta o classificador aos dados.
- `predict_labels` - recebe como parâmetro um classificador ajustado, atributos e rótulo alvo e faz estimativas utilizando a pontuação do F<sub>1</sub>.
- `train_predict` - recebe como entrada um classificador, e dados de treinamento e teste, e executa `train_clasifier` e `predict_labels`.
 - Essa função vai dar a pontuação F<sub>1</sub> tanto para os dados de treinamento como para os de teste, separadamente.

In [7]:
def train_classifier(clf, X_train, y_train):
    ''' Ajusta um classificador para os dados de treinamento. '''
    
    # Inicia o relógio, treina o classificador e, então, para o relógio
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Imprime os resultados
    print "O modelo foi treinado em {:.4f} segundos".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Faz uma estimativa utilizando um classificador ajustado baseado na pontuação F1. '''
    
    # Inicia o relógio, faz estimativas e, então, o relógio para
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Imprime os resultados de retorno
    print "As previsões foram feitas em {:.4f} segundos.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Treina e faz estimativas utilizando um classificador baseado na pontuação do F1. '''
    
    # Indica o tamanho do classificador e do conjunto de treinamento
    print "Treinando um {} com {} pontos de treinamento. . .".format(clf.__class__.__name__, len(X_train))
    
    # Treina o classificador
    train_classifier(clf, X_train, y_train)
    
    # Imprime os resultados das estimativas de ambos treinamento e teste
    print "Pontuação F1 para o conjunto de treino: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "Pontuação F1 para o conjunto de teste: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementação: Métricas de Desempenho do Modelo
Com as funções acima, você vai importar os três modelos de aprendizagem supervisionada de sua escolha e executar a função `train_prediction` para cada um deles. Lembre-se de que você vai precisar treinar e usar cada classificador para três diferentes tamanhos de conjuntos de treinamentos: 100, 200 e 300 pontos. Então você deve ter 9 saídas diferentes abaixo – 3 para cada modelo utilizando cada tamanho de conjunto de treinamento. Na célula de código a seguir, você deve implementar o seguinte:
- Importe os três modelos de aprendizagem supervisionada que você escolheu na seção anterior.
- Inicialize os três modelos e armazene eles em `clf_A`, `clf_B` e `clf_C`.
 - Defina um `random_state` para cada modelo, se a opção existir.
 - **Nota:** Utilize as configurações padrão para cada modelo – você vai calibrar um modelo específico em uma seção posterior.
- Crie diferentes tamanhos de conjuntos de treinamento para treinar cada modelo.
 - *Não embaralhe e distribua novamente os dados! Os novos pontos de treinamento devem ser tirados de `X_train` e `y_train`.*
- Treine cada modelo com cada tamanho de conjunto de treinamento e faça estimativas com o conjunto de teste (9 vezes no total).  
**Nota:** Três tabelas são fornecidas depois da célula de código a seguir, nas quais você deve anotar seus resultados.

In [8]:
# TODO: Importe os três modelos de aprendizagem supervisionada do sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# TODO: Inicialize os três modelos
clf_A = LogisticRegression()
clf_B = KNeighborsClassifier(n_neighbors=5, weights='distance')
clf_C = RandomForestClassifier(n_estimators=100, min_samples_leaf=3, random_state=0)

# TODO: Configure os tamanho dos conjuntos de treinamento
X_train_100, _, y_train_100, _ = train_test_split(X_all, y_all, train_size=100, random_state=0)

X_train_200, _, y_train_200, _ = train_test_split(X_all, y_all, train_size=200, random_state=0)

X_train_300, _, y_train_300, _ = train_test_split(X_all, y_all, train_size=300, random_state=0)

# TODO: Executar a função 'train_predict' para cada classificador e cada tamanho de conjunto de treinamento
# 1. Logistic Regression
print '---'
print '1. Logistic Regression'
print '\nTrain size = 100 \n'
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
print '\nTrain size = 200 \n'
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
print '\nTrain size = 300 \n'
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)
print

# 1. Logistic Regression
print '---'
print '2. KNN'
print '\nTrain size = 100 \n'
train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
print '\nTrain size = 200 \n'
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
print '\nTrain size = 300 \n'
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)
print

# 3. Random Forest
print '---'
print '3. Random Forest'
print '\nTrain size = 100 \n'
train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
print '\nTrain size = 200 \n'
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
print '\nTrain size = 300 \n'
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)
print

---
1. Logistic Regression

Train size = 100 

Treinando um LogisticRegression com 100 pontos de treinamento. . .
O modelo foi treinado em 0.0100 segundos
As previsões foram feitas em 0.0140 segundos.
Pontuação F1 para o conjunto de treino: 0.9412.
As previsões foram feitas em 0.0010 segundos.
Pontuação F1 para o conjunto de teste: 0.7385.

Train size = 200 

Treinando um LogisticRegression com 200 pontos de treinamento. . .
O modelo foi treinado em 0.0060 segundos
As previsões foram feitas em 0.0010 segundos.
Pontuação F1 para o conjunto de treino: 0.8710.
As previsões foram feitas em 0.0010 segundos.
Pontuação F1 para o conjunto de teste: 0.7820.

Train size = 300 

Treinando um LogisticRegression com 300 pontos de treinamento. . .
O modelo foi treinado em 0.0070 segundos
As previsões foram feitas em 0.0010 segundos.
Pontuação F1 para o conjunto de treino: 0.8381.
As previsões foram feitas em 0.0000 segundos.
Pontuação F1 para o conjunto de teste: 0.7910.

---
2. KNN

Train size = 10

### Resultados em tabelas
Edite a célula abaixo e veja como a tabela pode ser desenhada em [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). Você deve salvar seus resultados abaixo nas tabelas fornecidas.

** Classificador 1 - Logistic Regression**  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento | Tempo de Estimativa (teste) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100                                |        0.0040 s      |           0.0000 s          |            0.9412          |  0.7385              |
| 200                                |        0.0040 s      |           0.0010 s          |            0.8710          |  0.7820              |
| 300                                |        0.0060 s      |           0.0010 s          |            0.8381          |  0.7910              |

** Classificador 2 - 5-NN** 

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento | Tempo de Estimativa (teste) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100                                |        0.0020 s      |           0.0040 s          |            1.0000          |  0.7660              |
| 200                                |        0.0020 s      |           0.0060 s          |            1.0000          |  0.7376              |
| 300                                |        0.0020 s      |           0.0080 s          |            1.0000          |  0.7482              |

** Classificador 3 - Random Forest**  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento | Tempo de Estimativa (teste) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100                                |        0.4320 s      |           0.1030 s          |            0.9125          |  0.7867              |
| 200                                |        0.1310 s      |           0.1370 s          |            0.9161          |  0.7632              |
| 300                                |        0.1330 s      |           0.1230 s          |            0.9321          |  0.7945              |

## Escolhendo o Melhor Modelo
Nesta seção final, você irá escolher dos três modelos de aprendizagem supervisionada o *melhor* para utilizar os dados dos estudantes. Você então executará um busca em matriz otimizada para o modelo em todo o conjunto de treinamento (`X_train` e `y_train`) ao calibrar pelo menos um parâmetro, melhorando em comparação a pontuação F<sub>1</sub> do modelo não calibrado. 

### Questão 3 - Escolhendo o Melhor Modelo
*Baseando-se nos experimentos que você executou até agora, explique em um ou dois parágrafos ao conselho de supervisores qual modelo que você escolheu como o melhor. Qual modelo é o mais apropriado baseado nos dados disponíveis, recursos limitados, custo e desempenho?*

**Resposta: ** O modelo com *score* F1 maior foi o Random Forest com tamanho 300 de treinamento. No entanto, este modelo demora mais de 20 vezes para ser treinamento e 120 vezes mais para fazer as previsões do que a regressão logística, que obteve um F1 score apenas .3% inferior. Portanto o modelo escolhido é a regressão logística.

### Questão 4 – O Modelo para um Leigo
*Em um ou dois parágrafos, explique para o conselho de supervisores, utilizando termos leigos, como o modelo final escolhido deve trabalhar. Tenha certeza que você esteja descrevendo as melhores qualidades do modelo, por exemplo, como o modelo é treinado e como ele faz uma estimativa. Evite jargões técnicos ou matemáticos, como descrever equações ou discutir a implementação do algoritmo.*

**Resposta: ** O modelo escolhido leva em consideração todas as variáveis que coletamos de cada aluno e com base nos dados históricos eles nos retorna a probabilidade de um dado aluno se graduar ou não. Podemos definir em conjunto uma probabilidade de corte para atuação ativa e uma faixa para observação de alunos.

Para que o modelo preveja casos futuros, é importante que continuemos a recolher os mesmos dados que temos hoje, tanto socioeconômicos, quanto das faltas. Assim, basta mantermos a base atualizada e o modelo nos dará a probabilidade do aluno em questão se formar ou não e, então, podemos agir.

### Implementação: Calibrando o Modelo (_Tuning_)
Calibre o modelo escolhido. Utilize busca em matriz (`GridSearchCV`) com, pelo menos, um parâmetro importante calibrado com, pelo menos, 3 valores diferentes. Você vai precisar utilizar todo o conjunto de treinamento para isso. Na célula de código abaixo, você deve implementar o seguinte:
- Importe [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) e [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Crie o dicionário de parâmetros que você deseja calibrar para o modelo escolhido.
 - Examplo: `parameters = {'parameter' : [list of values]}`.
- Inicialize o classificador que você escolheu e armazene-o em `clf`.
- Crie a função de pontuação F<sub>1</sub> utilizando `make_scorer` e armazene-o em `f1_scorer`.
 - Estabeleça o parâmetro `pos_label` para o valor correto!
- Execute uma busca em matriz no classificador `clf` utilizando o `f1_scorer` como método de pontuação e armazene-o em `grid_obj`.
- Treine o objeto de busca em matriz com os dados de treinamento (`X_train`, `y_train`) e armazene-o em `grid_obj`.

In [9]:
# TODO: Importe 'GridSearchCV' e 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

# TODO: Crie a lista de parâmetros que você gostaria de calibrar
parameters = {'penalty': ['l1', 'l1'], 
              'C': [0.1, 0.5, 1.0, 2.0, 10.0], 
              'fit_intercept': ['True', 'False']}

# TODO: Inicialize o classificador
clf = LogisticRegression()

# TODO: Faça uma função de pontuação f1 utilizando 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Execute uma busca em matriz no classificador utilizando o f1_scorer como método de pontuação
grid_obj = GridSearchCV(clf, 
                        param_grid=parameters,
                        scoring=f1_scorer)

# TODO: Ajuste o objeto de busca em matriz para o treinamento de dados e encontre os parâmetros ótimos
grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Reporte a pontuação final F1 para treinamento e teste depois de calibrar os parâmetros
print 'Os parametros escolhidos foram {}'.format(clf)
print "O modelo calibrado tem F1 de {:.4f} no conjunto de treinamento.".format(predict_labels(clf, X_train, y_train))
print "O modelo calibrado tem F1 de {:.4f} no conjunto de teste.".format(predict_labels(clf, X_test, y_test))

C:\Anaconda3\envs\py27\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Anaconda3\envs\py27\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Os parametros escolhidos foram LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept='True',
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
As previsões foram feitas em 0.0000 segundos.
O modelo calibrado tem F1 de 0.8267 no conjunto de treinamento.
As previsões foram feitas em 0.0010 segundos.
O modelo calibrado tem F1 de 0.7917 no conjunto de teste.


### Questão 5 - Pontuação F<sub>1</sub> Final
*Qual é a pontuação F<sub>1</sub> do modelo final para treinamento e teste? Como ele se compara ao modelo que não foi calibrado?*

**Resposta: ** A pontuação F1 final do modelo para treinamento foi de .8267 (pior em .0114 do não calibrado) e para teste .7917 (melhor em .0007 do não calibrado)

> **Nota**: Uma vez que você completou todas as implementações de código e respondeu todas as questões acima com êxito, você pode finalizar seu trabalho exportando o iPython Nothebook como um document HTML. Você pode fazer isso utilizando o menu acima e navegando para  
**File -> Download as -> HTML (.html)**. Inclua a documentação final junto com o notebook para o envio do seu projeto.